In [16]:
import numpy as np
import math
import matplotlib as plt
from statistics import mean
import pandas as pd
import scipy.stats as st
import networkx as nx


class AgentModel:
    
    class Donators:                        #  В модели 100 спонсоров.
        '''
         В каждом раунде спонсор отдаёт целиком 1 монетку j-той НКО, от которой он получил 
         самый сильный сигнал self.signal[i][j]. Значение сигнала  self.signal[i][j] - это значение, 
         сгенерированное из нормального распределения, дисперсия которого равна 9, а матожидание
         складывается из бюджета j-той организации на медицину и рекламу и коэффициентов 
         предпочтения i-того спонсора.
         
         self.signal[i][j] ∼ N(a, 9)
         a = self.med_coef[i] * medbudjet[j] + self.adv_coef[i] * advbudjet[j]
        '''
        
        def __init__(self):
            self.data = pd.DataFrame({'budget' : [0] * 100, 'choice' : [0] * 100, 'prognosis' : [0.] * 100,
                                     '0-sig' : [0.] * 100, '1-sig' : [0.] * 100, '2-sig' : [0.] * 100, '3-sig' : [0.] * 100, '4-sig' : [0.] * 100,
                                     '5-sig' : [0.] * 100, '6-sig' : [0.] * 100, '7-sig' : [0.] * 100, '8-sig' : [0.] * 100, '9-sig' : [0.] * 100,
                                     '0' : [0.] * 100, '1' : [0.] * 100, '2' : [0.] * 100, '3' : [0.] * 100, '4' : [0.] * 100,
                                     '5' : [0.] * 100, '6' : [0.] * 100, '7' : [0.] * 100, '8' : [0.] * 100, '9' : [0.] * 100})
            #  0-9 - это Сигналы от НКО
            
            #self.med_coef = 0              #  Коэффициент предпочтения медицины 
            #self.adv_coef = 0              #  Коэффициент предпочтения рекламы
            #self.signal = 0                #  Массив из сигналов. У каждого из 100 доноров
                                           #  есть 10 сигналов от организаций.
            #self.prev_donations = 0        #  Сколько раз спонсор жертвовал организациям
                                           #  в предыдущих ранудахю
            #self.reset()
            
        def reset(self):
            self.data = pd.DataFrame({'budget' : [0] * 100, 'choice' : [0] * 100, 'prognosis' : [0.] * 100,
                                     '0-sig' : [0.] * 100, '1-sig' : [0.] * 100, '2-sig' : [0.] * 100, '3-sig' : [0.] * 100, '4-sig' : [0.] * 100,
                                     '5-sig' : [0.] * 100, '6-sig' : [0.] * 100, '7-sig' : [0.] * 100, '8-sig' : [0.] * 100, '9-sig' : [0.] * 100,
                                     '0' : [0.] * 100, '1' : [0.] * 100, '2' : [0.] * 100, '3' : [0.] * 100, '4' : [0.] * 100,
                                     '5' : [0.] * 100, '6' : [0.] * 100, '7' : [0.] * 100, '8' : [0.] * 100, '9' : [0.] * 100})
            
    class Patients:                        #  В модели 100 пациентов.

        '''
         i-тый пациент в каждом раунде обращается за помощью в j-тую НКО, от которой он получил 
         самый сильный сигнал self.signal[i][j]. При этом, если пациент уже обращался в 
         j-тую организацию на прошлых раундах, то в этом раунде он туда не пойдёт.
        '''
        
        def __init__(self, graph_status = False):
            self.data = pd.DataFrame({'budget' : [0] * 100, 'choice' : [0] * 100, 'prognosis' : [0.] * 100,
                                     '0' : [False] * 100, '1' : [False] * 100, '2' : [False] * 100, '3' : [False] * 100, '4' : [False] * 100,
                                     '5' : [False] * 100, '6' : [False] * 100, '7' : [False] * 100, '8' : [False] * 100, '9' : [False] * 100,})
            
            self.graph = nx.gnp_random_graph(100, 0.1)

            
            # 0-9 Какие организации посетил пациент?

            
            #self.choice = 0                #  Какую организацию выбирают пациенты в текущем раунде?
            #self.p_survival = 0            #  Вероятность излечения без помощи организации.
            #self.visited = 0               #  Какие организации уже посетили пациенты?
            #self.budget = 0                #  Деньги, выделенные на конкретного пациента в текущем раунде.
            
            self.count_cured = 0           #  Сколько всего вылечено пациентов?
            
            #self.reset()

        def reset(self, graph_status = False):
            self.data = pd.DataFrame({'budget' : [0] * 100, 'choice' : [0] * 100, 'prognosis' : [0.] * 100,
                                     '0' : [False] * 100, '1' : [False] * 100, '2' : [False] * 100, '3' : [False] * 100, '4' : [False] * 100,
                                     '5' : [False] * 100, '6' : [False] * 100, '7' : [False] * 100, '8' : [False] * 100, '9' : [False] * 100,})
            #  
        
        def new_patient(self, i):
            pass
            
    class Organizations:

        '''
        Организация получает бюджет от спонсоров. Если бюджет нулевой, то НКО выбывает из игры. 
        Если она получает ненулевой бюджет, то часть денег она тратит на административные расходы, 
        а часть на программную деятельнотсь.
        self.med_coef[j] * self.budget[j], а часть на рекламу self.adv_coef[j] * self.budget[j]
        '''
        
        def __init__(self, diag_count = 1):
            self.data = pd.DataFrame({'budget' : [0] * 10, 'medical_expenses' : [0] * 10, 'administrative_expenses' : [0] * 10, 
                                      'is_lose' : [False] * 10, 'laps' : [0] * 10, 'saved_lifes' : [0] * 10, 'patients_count' : [0] * 10})
            
            self.tax = 0                  #  Процент налога на рекламу.


            #self.budget = 0              #  Общий бюджет каждой организации в текущем раунде
            #self.medical_expenses = 0
            #self.administrative_expenses = 0
            #self.med_coef = 0            #  Коэффициент трат на медицину.
            #self.is_lose = 0             #  Выбыла ли организация?
            #self.prev_patients = 0
            #self.patients = 0            #  Индексы пациентов, которые поступили в 
                                          #  организацию в текущем раунде.
            #self.laps = 0                #  Сколько кругов продержалась организация?
            #self.saved_lives = 0         #  Сколько жизней спасла организация?
            #self.all_patients = 0        #  Сколько пациентов поступило в организацию
                                          #  на протяжении всей симцляции?
            
            #self.diag_count = diag_count
        
            #self.reset(tax)
            
        def reset(self, new_tax = 0):
            self.data = pd.DataFrame({'budget' : [0] * 10, 'medical_expenses' : [0] * 10, 'administrative_expenses' : [0] * 10, 
                                      'is_lose' : [False] * 10, 'laps' : [0] * 10, 'saved_lifes' : [0] * 10, 'patients_count' : [0] * 10})
            self.tax = new_tax

    '''
    Конец описания подклассов.
    '''
    
    def __init__(self, graph_status = False, diag_count = 1):
        self.donators = AgentModel.Donators()
        self.patients = AgentModel.Patients(graph_status, diag_count)
        self.organizations = AgentModel.Organizations(diag_count)
        
    def donation(self):
        df = self.donators.data[['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
        indices = df.idxmax(axis="columns")
        
        for i in range(100): 
            ind = np.argmax(self.donators.signal[i]) 
            self.donators.prev_donations[i][ind] += 1
            self.organizations.budget[ind] += 1        
            
    def budget_med_or_adv(self):
        for i in range(10):
            if self.organizations.budget[i] == 0:
                self.organizations.is_lose[i] = True
            else:
                self.organizations.laps[i] += 1
                self.organizations.med_budget[i] = self.organizations.budget[i] * self.organizations.med_coef[i]
                self.organizations.adv_budget[i] = (self.organizations.budget[i] - self.organizations.med_budget[i]) * (1 - self.organizations.tax)
                self.organizations.budget[i] = 0

    def check_friends(self, patient, org):
        friends_count = 0
        surv = 0
        #  если у нас новый пациент, то непонятно, как смотреть, вылечились ли его друзбя где-то
        #  надо смотреть, что они были в предыдущем круге
        #  он дружит с человеком, который раньше был на этом месте? или с новым пациентом?
        
        #if self.patients.who_save[patient] != -1:
        #    friends = np.array([friend for friend in range(100) if (patients.graph[patient][friend] and patients.who_save[friend] == -1)])
        #else:
        friends = np.nonzero(self.patients.prev_graph[patient])[0]
        for friend in friends:
            if self.patients.prev_visited[patient][org]:
                friends_count += 1
                if self.patients.who_save[friend] == org:
                    surv += 1    
        return (friends_count > 2 and (surv / friends_count) < 0.5) #  возможно, взять коэффицент не 0.5


    def choose_organization(self):          
        for i in range(100):
            for j in range(10):
                if self.patients.visited[i][j] or self.organizations.is_lose[j]:
                    self.patients.signal[i][j] = -100
                    continue
                a =  self.organizations.med_budget[j] * self.patients.med_coef[i] + self.organizations.adv_budget[j] * self.patients.adv_coef[i]
                #  Если есть граф
                if self.patients.graph_status and self.patients.who_save[i] == -1 and self.check_friends(i, j): 
                    self.patients.signal[i][j] = -100
                    continue
                self.patients.signal[i][j] = np.random.normal(a, 9)
                
            ind = np.argmax(self.patients.signal[i])
            if self.patients.signal[i][ind] == -100:
                for j in range(10):
                    self.patients.visited[i][j] = False
                    if not self.organizations.is_lose[j]:
                        a =  self.organizations.med_budget[j] * self.patients.med_coef[i] + self.organizations.adv_budget[j] * self.patients.adv_coef[i]
                        self.patients.signal[i][j] = np.random.normal(a, 9, 1)[0]
                ind = np.argmax(self.patients.signal[i])
                   
            self.patients.visited[i][ind] = True
            self.patients.choice[i] = ind
            self.organizations.all_patients[ind] += 1
            self.organizations.patients[ind].append(i)
                                                                                                                                                
    def budget_between_patient(self): #  Может ли организация не принять?
        for i in range(10):
            self.organizations.prev_patients[i] = self.organizations.patients[i]  #  Сохраняем
            
            if (self.organizations.is_lose[i]):
                continue
            if len(self.organizations.patients[i]) > 0:
                patient_budget = self.organizations.med_budget[i] / len(self.organizations.patients[i])
            else:
                patient_budget = 0
                                                                    
            for patient in (self.organizations.patients[i]):
                self.patients.budget[patient] = patient_budget          
            self.organizations.patients[i] = []
        
    def survival(self):                 #  поменять коэффициенты в ???
        if  self.patients.graph_status:
            self.patients.prev_graph = self.patients.graph   #  старые связи с пациентами (часть пациентов в новом кругу может быть заменена новыми)
            self.patients.prev_visited = self.patients.visited
        for i in range(100):            #  logit ~ (0.5, 1) - p выживаемости. 
            s_i = (1 / (1 + math.pow(math.e, -(-0.5 + self.patients.p_survival[i] + self.patients.budget[i]) ))) - 0.2  #  коэффициент эффективности трат
            if np.random.binomial(1, s_i): #  Пациент вылечился (Биномиальное распределение)
                self.patients.count_cured += 1
                self.patients.new_patient(i)
                self.organizations.saved_lifes[self.patients.diagnosis[i]][self.patients.choice[i]] += 1
                self.patients.who_save[i] == self.patients.choice[i]
            else:
                self.patients.who_save[i] == -1
        
    def set_new_preferences(self):
        self.organizations.data.loc['budget'] = 0
        
    
    def lap(self):
        self.donation()        
        self.budget_med_or_adv()
        self.choose_organization()
        self.budget_between_patient()
        self.survival()
        self.set_new_preferences()
        
    def simulation(self):
        self.donators.reset()
        self.patients.reset()
        self.organizations.reset(self.organizations.tax)
        i = 0
        while True:
            i += 1
            self.lap()
            if sum(self.organizations.is_lose == True) == 9:
                winner_ind = 0
                for j in range(10):
                    if not self.organizations.is_lose[j]:
                        winner_ind = j
                        break
                return i, self.organizations.laps, self.organizations.med_coef, self.organizations.saved_lifes, self.organizations.all_patients, winner_ind
            
   

In [17]:
model = AgentModel()
model.organizations.data

,budget,medical_expenses,administrative_expenses,is_lose,laps,saved_lifes,patients_count
0,0,0,0,False,0,0,0
1,0,0,0,False,0,0,0
2,0,0,0,False,0,0,0
3,0,0,0,False,0,0,0
4,0,0,0,False,0,0,0
5,0,0,0,False,0,0,0
6,0,0,0,False,0,0,0
7,0,0,0,False,0,0,0
8,0,0,0,False,0,0,0
9,0,0,0,False,0,0,0


In [13]:
model.organizations.data.dtypes

budget                     int64
medical_expenses           int64
administrative_expenses    int64
is_lose                     bool
laps                       int64
saved_lifes                int64
patients_count             int64
dtype: object

In [14]:
model.donators.data

,budget,choice,prognosis,0,1,2,3,4,5,6,7,8,9
0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Итоги встречи 1

**Спонсоры**

- спонсоры ≠ доноры. Спонсоры – те, кто хочет что-то взамен. Таких очень мало.

**НКО**

- Есть ограничение в 20% по тратам на уставные расходы. И все «траты на себя» воспринимаются донорами крайне негативно. Деньги на крупные проекты, не связанные напрямую с закупкой лекарств на конкретного человека, как правило, дают государство и крупные фонды.
- Фонды себя не рекламируют больным. Больные найдут фонды сами. Рекламируются бесплатно.
- НКО проверяют пациентов на честность. Смотрят диагнозы и прогнозы (слепое рецензирование). Если прогноз плохой, то могут отказать. Но некоторые НКО стараются брать всех в надежде на лучшее. Все зависит от решений основателя НКО.
- Эффект «друг в АП» влияет на то, дадут ли НКО билборды.
- Государство оплачивает лечение пациента, а НКО оплачивает переезд, реабилитацию после лечения, собирает пакеты документов для государства.
- НКО общаются друг с другом и могут передать пациента
- НКО для детей гораздо больше, чем НКО для взрослых
- Бывают специализированные НКО, бывают общие
- Часто директор тянет на себе кучу задач. Не хватает средств для зарплаты сотрудникам, прописывают расходы через программную деятельность. Не получается устроить на постоянную ставку.
- При этом обучение обычно предоставляется бесплатно. Но нет системного обучения.
- Нельзя привлекать сотрудников за счет корпоративных штучек: доступ к спортзалам и т.д.
- Деятельнсть НКО очень жестко контролируется. Много отчетности. Можно почитать законы, регулирующие НКО, чтобы разобраться.

**Пациенты**
- Пациенты обычно обращаются в НКО со смертельными диагнозами. И речь идёт не об излечении, а об улучшении прогноза, повышении качества жизни 
- Пациент может обратиться сразу в несколько организаций. И случается, что ему дают деньги сразу несколько НКО.


**Вопросы.**

- а существует ли статистика выживания пациентов? Можно сравнить статистику в организациях, которые берут всех и которые берут с хорошим прогнозом.
- Если есть НКО, которые берут пациентов, не обращая внимания на прогноз, то каким принципом они руководствуются, если бюджета не хватает? Они берут в порядке очереди?

**Идеи**

- Четвертый рынок агентов?


1. Улучшаем параметры модели
2. Насколько эффективен этот закон?
3. Четвёртый агент. Рынок труда. Отдача от качества кадров.
- Можно нанять молодых сотрудников. Они учатся. Могут стать лучшими сотрудниками. Параметр опыта - качество персонала
- Можно нанять дорогих сотрудников. Какое-то время будут проигрывать.
- Опыт - это параметр вылечившихся + опыт специалистов. Коэффициент эффективности траты бюджета. Умножаем лечение на 1.1
- Лгущие пациенты. Связи между организациями. Эффект слухов между пациентами. Сеть между. У организации есть оптимизационная задача. Если прогноз плохой, то они могут и отказать. У каждого пациента есть функция выживаемости.
- Предварительный скрининг пациентов, которые пошли сразу в 2 организации.

`Метод симплекса (алгоритм)` - проверка. Линейное программирование. Ещё спросить у АА. 

- рюкзак.

- Можно оперировать не вероятностью выживания, а стоимостью и продолжительностью реабилитации.

- Как связаны организация. Топология сети. Как можно моделировать сети с разной топологией. 
- Оставить сигнал, основанный на медицине.
- Интерпретиурем раунд как месяц.
- Убрать налог. Сделать ограничение. Пациенты получают только медицинский сигнал. А спонсоры получают общий сигнал.

### Что рассказать
- Это важный сектор. Но мы об этом секторе мало знаем. Есть данные опросы. Но нет четкого систематического понимания, как работает рынок. Как различаются параметры рынка.
- Перспективы. Рынок труда.
- Можно сделать с описанием нюансов, моделей.